<a href="https://colab.research.google.com/github/utensil/llm-playground/blob/main/notebooks/axolotl/runpod/axolotl-falcon-40b-qlora-deepspeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- https://jupyterlab.readthedocs.io/en/stable/user/commands.html#commands-list -->
<button data-commandLinker-command="apputils:change-theme" data-commandlinker-args='{"theme": "JupyterLab Dark"}' href="#">Dark theme</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/content/llm-playground/"}' href="#">llm-playground</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/"}' href="#">axolotl</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/content/llm-playground/notebooks/axolotl/runpod"}' href="#">Runpod notebooks</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/examples"}' href="#">axolotl configs</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/content/llm-playground/storage"}' href="#">Storage</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/content/axolotl-trained"}' href="#">axolotl-trained</button>
<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path": "/workspace/axolotl/examples/falcon/config-40b-qlora.yml"}' href="#">Edit qlora config</button>
<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path": "/workspace/axolotl/ds_config.json"}' href="#">Edit ds config</button>

# Finetuning falcon-40b

- Axolotl+QLoRA
- minotaur datasets
- deepspeed ZeRO 3 8xGPU

Entry script:

https://github.com/utensil/llm-playground/blob/main/scripts/entry/ax_lite.sh

Mount volume disk to `/content`

Save to:

- repo: `utensil/llm-playground`
- path: `notebooks/axolotl/runpod/axolotl-falcon-40b-qlora-deepspeed.ipynb`


## Prepare

### Set HF Cache

In [ ]:
# %env HF_DATASETS_CACHE

In [ ]:
#%env TRANSFORMERS_CACHE

In [ ]:
!rm -rf /root/.cache

In [ ]:
!mkdir -p /content/cache

In [ ]:
!ln -s /content/cache /root/.cache

### Speed up model download

In [ ]:
!apt-get update
!apt-get install -y aria2
!git lfs install
!pip install requests huggingface_hub

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [458 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1181 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [459 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [866 kB]
Fetched 3301 kB in 2s (2199 kB/s)                        
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2

In [ ]:
#%cd /content/

In [ ]:
#!git clone https://github.com/utensil/llm-playground

In [ ]:
%cd /content/llm-playground

/content/llm-playground


In [ ]:
!python /content/llm-playground/helper/download-model.py tiiuae/falcon-40b
# /content/llm-playground/models/tiiuae_falcon-40b

Working directory changed to: /content/llm-playground/helper/..
Skipping: .gitattributes
Running: aria2c -c -x 16 -s 16 -k 1M https://huggingface.co/tiiuae/falcon-40b/resolve/main/README.md -d models/tiiuae_falcon-40b -o README.md
Running: aria2c -c -x 16 -s 16 -k 1M https://huggingface.co/tiiuae/falcon-40b/resolve/main/config.json -d models/tiiuae_falcon-40b -o config.json
Running: aria2c -c -x 16 -s 16 -k 1M https://huggingface.co/tiiuae/falcon-40b/resolve/main/configuration_RW.py -d models/tiiuae_falcon-40b -o configuration_RW.py
Running: aria2c -c -x 16 -s 16 -k 1M https://huggingface.co/tiiuae/falcon-40b/resolve/main/generation_config.json -d models/tiiuae_falcon-40b -o generation_config.json
Running: aria2c -c -x 16 -s 16 -k 1M https://huggingface.co/tiiuae/falcon-40b/resolve/main/modelling_RW.py -d models/tiiuae_falcon-40b -o modelling_RW.py
Running: aria2c -c -x 16 -s 16 -k 1M https://huggingface.co/tiiuae/falcon-40b/resolve/main/pytorch_model-00001-of-00009.bin -d models/tiiua

In [ ]:
# Downloading shards: 100%|████████████████████████| 9/9 [19:19<00:00, 128.85s/it]
# Loading checkpoint shards: 100%|██████████████████| 9/9 [01:30<00:00, 10.08s/it]
# (OK):download completed.
# 100%|████████████████████████████████████████████████████████████████████| 18/18 [01:44<00:00,  5.81s/it]

### HF Login

In [ ]:
# For axolotl push_dataset_to_hub
import os
from huggingface_hub import notebook_login, login
# Colab:
# notebook_login()
# RunPod:
login(os.environ.get("HUGGINGFACE_TOKEN"), add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Update axolotl

In [ ]:
%cd /workspace/

In [ ]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl axolotl-update

In [ ]:
!cp -r axolotl-update/* axolotl

In [ ]:
%cd /workspace/axolotl

In [ ]:
!git status

In [ ]:
!ds_report

In [ ]:
!pip install -e .

In [ ]:
!pip list|grep torch

### Init Storage

In [ ]:
!python /content/llm-playground/helper/storage.py utensil/axolotl-trained /content/ -m

Cloning https://huggingface.co/utensil/axolotl-trained into local empty directory.


In [ ]:
!ls /content/axolotl-trained

### Reinstall PyTorch with CUDA 11.8 (optional)

In [ ]:
!pip3 install -U torch --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 3.3 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 2.0.1+cu118 which is incompatible.


In [ ]:
!pip list|grep torch

torch                    2.0.1+cu118
torchaudio               2.0.1+cu118
torchvision              0.15.2


### Reinstall deepspeed (optional)

In [ ]:
!ds_report

Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
async_io ............... [YES] ...... [OKAY]
cpu_adagrad ............ [YES] ...... [OKAY]
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [YES] ...... [OKAY]
fused_lamb ............. [YES] ...... [OKAY]
quantizer .............. [YES] ...... [OKAY]
random_ltd ............. [YES] ...... [OKAY]
 [WARNING]  sparse_attn requires a torch vers

In [ ]:
!yes|pip uninstall deepspeed

In [ ]:
!TORCH_CUDA_ARCH_LIST="3.5;5.0;6.0;6.1;7.0;7.5;8.0;8.6+PTX" DS_BUILD_OPS=1 DS_BUILD_SPARSE_ATTN=0 pip install deepspeed --global-option="build_ext" --global-option="-j8" # --global-option="bdist_wheel"

In [ ]:
!pip install deepspeed

In [ ]:
!ds_report

### Init Configs

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
# Try no config
# !accelerate config default

In [ ]:
%%writefile ds_config.json
{
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": true
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 0,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 0,
        "stage3_max_reuse_distance": 0,
        "stage3_gather_16bit_weights_on_model_save": true
    },
   "bf16": {
        "enabled": "auto"
    },
    "fp16": {
        "enabled": "auto",
        "auto_cast": false,
        "loss_scale": 0,
        "initial_scale_power": 32,
        "loss_scale_window": 1000,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
          "lr": "auto",
          "betas": "auto",
          "eps": "auto",
          "weight_decay": "auto"
        }
    },
    "scheduler": {
      "type": "WarmupDecayLR",
      "params": {
        "total_num_steps": "auto",
        "warmup_min_lr": "auto",
        "warmup_max_lr": "auto",
        "warmup_num_steps": "auto"
       }
    },
    "gradient_accumulation_steps": "auto",
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}

Overwriting ds_config.json


In [ ]:
%%writefile examples/falcon/config-40b-qlora.yml
# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"

dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all linear layers of the base model
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project: falcon-qlora
wandb_watch:
wandb_run_id:
wandb_log_model:
output_dir: /content/axolotl-trained/falcon-qlora-40b-gsm8k/

# QLoRA paper Table 9
# - 16 for 7b & 13b
# - 32 for 33b, 64 for 64b
# Max size tested on A6000
# - 7b: 40
# - 40b: 4
# decrease if OOM, increase for max VRAM utilization
micro_batch_size: 1
gradient_accumulation_steps: 1
num_epochs: 3
# Optimizer for QLoRA
# optimizer: paged_adamw_32bit
torchdistx_path:
# lr_scheduler: cosine
# QLoRA paper Table 9
# - 2e-4 for 7b & 13b
# - 1e-4 for 33b & 64b
learning_rate: 0.0002
train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: true
gradient_checkpointing: true
# stop training after this many evaluation losses have increased in a row
# https://huggingface.co/transformers/v4.2.2/_modules/transformers/trainer_callback.html#EarlyStoppingCallback
early_stopping_patience: 3
resume_from_checkpoint:
auto_resume_from_checkpoints: true
local_rank:
logging_steps: 1
xformers_attention: true
flash_attention:
gptq_groupsize:
gptq_model_v1:
warmup_steps: 10
eval_steps: 5
save_steps: 10
debug:
deepspeed:
weight_decay: 0.01
fsdp:
fsdp_config:
special_tokens:
  pad_token: "<|endoftext|>"
  bos_token: ">>ABSTRACT<<"
  eos_token: "<|endoftext|>"

Writing examples/falcon/config-40b-qlora.yml


In [ ]:
%env ACCELERATE_USE_DEEPSPEED=true

env: ACCELERATE_USE_DEEPSPEED=true


In [ ]:
%%writefile scripts/ft.py
import os
from pathlib import Path
import fire
import logging
import finetune
from axolotl.utils.trainer import setup_trainer as setup_trainer_orig

logging.basicConfig(level=os.getenv("LOG_LEVEL", "INFO"))

def train_ex(
    config: Path = Path("configs/"),
    prepare_ds_only: bool = False,
    **kwargs,
):
  logging.info('train_ex before')
  finetune.train(config, prepare_ds_only, **kwargs)
  logging.info('train_ex after')

def setup_trainer_ex(cfg, train_dataset, eval_dataset, model, tokenizer):
  logging.info('setup_trainer_ex before')
  logging.info(f'cfg.some_config = {cfg.some_config}')
  trainer = setup_trainer_orig(cfg, train_dataset, eval_dataset, model, tokenizer)
  logging.info('setup_trainer_ex after')
  return trainer

finetune.setup_trainer = setup_trainer_ex

if __name__ == "__main__":
    fire.Fire(train_ex)

Writing scripts/ft.py


# Training

## #1 DS Rountine Attempt

In [ ]:
%env HF_HUB_DISABLE_PROGRESS_BARS=1

env: HF_HUB_DISABLE_PROGRESS_BARS=1


In [ ]:
%env ACCELERATE_USE_DEEPSPEED=true

env: ACCELERATE_USE_DEEPSPEED=true


In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"

dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B an

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.

## #2 ACC mbs 32 2xA100 minotaur OOM

In [ ]:
%env ACCELERATE_USE_DEEPSPEED=false

env: ACCELERATE_USE_DEEPSPEED=false


In [ ]:
%%writefile examples/falcon/config-40b-qlora.yml
# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: winglian/evals
    data_files:
      - hf/ARC-Challenge.jsonl
      - hf/ARC-Easy.jsonl
      - hf/riddle_sense.jsonl
    type: explainchoice:chat
  - path: winglian/evals
    data_files:
      - hf/gsm8k.jsonl
      - hf/winogrande.jsonl
    type: alpaca_chat.load_qa
  - path: winglian/evals
    data_files:
      - custom/n_task.jsonl
      - custom/misconceptions.jsonl
      - custom/context_insensitivity.jsonl
    type: alpaca_chat
  - path: camel-ai/math
    type: alpaca_chat.load_camel_ai
  - path: camel-ai/biology
    type: alpaca_chat.load_camel_ai
  - path: camel-ai/physics
    type: alpaca_chat.load_camel_ai
  - path: camel-ai/chemistry
    type: alpaca_chat.load_camel_ai
  - path: winglian/evals
    data_files:
      - custom/in_context_qa.jsonl
    type: context_qa
  - path: winglian/evals
    data_files:
      - custom/in_context_qa.jsonl
    type: context_qa.load_404
  - path: winglian/evals
    data_files:
      - custom/jokes_explained_500up.jsonl
    type: sharegpt_jokes
  - path: winglian/evals
    data_files:
      - custom/classify-self-chat.sharegpt.jsonl
      - custom/coding-self-chat.sharegpt.jsonl
      - custom/prose-gpt4.sharegpt.jsonl
      - custom/prose-rewrite-gpt4.sharegpt.jsonl
    type: sharegpt_simple.load_role
  - path: winglian/evals
    data_files:
      - openai/tldr.jsonl
    type: summarizetldr:chat
  - path: winglian/evals
    data_files:
      - hellaswag/hellaswag.jsonl
    type: explainchoice:chat
  - path: metaeval/ScienceQA_text_only
    type: concisechoice:chat
  - path: teknium/GPT4-LLM-Cleaned
    type: alpaca_chat
  - path: teknium/GPTeacher-General-Instruct
    data_files: gpt4-instruct-similarity-0.6-dataset.json
    type: gpteacher:chat
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/aqua_train.json
      - Chain-of-Thought/formatted_cot_data/creak_train.json
      - Chain-of-Thought/formatted_cot_data/ecqa_train.json
      - Chain-of-Thought/formatted_cot_data/esnli_train.json
      - Chain-of-Thought/formatted_cot_data/qasc_train.json
      - Chain-of-Thought/formatted_cot_data/qed_train.json
      - Chain-of-Thought/formatted_cot_data/sensemaking_train.json
      - Chain-of-Thought/formatted_cot_data/strategyqa_train.json
      - GPTeacher/Roleplay/formatted_roleplay-similarity_0.6-instruct-dataset.json
    type: alpaca_chat
  - path: ehartford/WizardLM_alpaca_evol_instruct_70k_unfiltered
    type: alpaca_chat
  - path: ehartford/wizard_vicuna_70k_unfiltered
    type: sharegpt:chat

dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all linear layers of the base model
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project: falcon-qlora
wandb_watch:
wandb_run_id:
wandb_log_model:
output_dir: /content/axolotl-trained/falcon-qlora-40b-gsm8k/

# QLoRA paper Table 9
# - 16 for 7b & 13b
# - 32 for 33b, 64 for 64b
# Max size tested on A6000
# - 7b: 40
# - 40b: 4
# decrease if OOM, increase for max VRAM utilization
micro_batch_size: 32
gradient_accumulation_steps: 1
num_epochs: 3
# Optimizer for QLoRA
optimizer: paged_adamw_32bit
torchdistx_path:
lr_scheduler: cosine
# QLoRA paper Table 9
# - 2e-4 for 7b & 13b
# - 1e-4 for 33b & 64b
learning_rate: 0.0002
train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: true
gradient_checkpointing: true
# stop training after this many evaluation losses have increased in a row
# https://huggingface.co/transformers/v4.2.2/_modules/transformers/trainer_callback.html#EarlyStoppingCallback
early_stopping_patience: 3
resume_from_checkpoint:
auto_resume_from_checkpoints: true
local_rank:
logging_steps: 1
xformers_attention: true
flash_attention:
gptq_groupsize:
gptq_model_v1:
warmup_steps: 10
eval_steps: 5
save_steps: 10
debug:
deepspeed:
weight_decay: 0.01
fsdp:
fsdp_config:
special_tokens:
  pad_token: "<|endoftext|>"
  bos_token: ">>ABSTRACT<<"
  eos_token: "<|endoftext|>"

Overwriting examples/falcon/config-40b-qlora.yml


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
!ds_report

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.

## #3 2xA100 mbs 16 OOM

In [ ]:
%env HF_HUB_DISABLE_PROGRESS_BARS=1

In [ ]:
%env ACCELERATE_USE_DEEPSPEED=false

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: winglian/evals
    data_files:
      - hf/ARC-Challenge.jsonl
      - hf/ARC-Easy.jsonl
      - hf/riddle_sense.jsonl
    type: explainchoice:chat
  - path: winglian/evals
    data_files:
      - hf/gsm8k.jsonl
      - hf/winogrande.jsonl
    type: alpaca_chat.load_qa
  - path: winglian/evals
    data_files:
      - custom/n_task.jsonl
      - custom/misconceptions.jsonl
      - custom/context_insensitivity.jsonl
    type: alpaca_chat
  - path: camel-a

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `2`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.

## #4 4xA100 mbs 16 OOM

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
%env ACCELERATE_USE_DEEPSPEED=false

env: ACCELERATE_USE_DEEPSPEED=false


In [ ]:
%env HF_HUB_DISABLE_PROGRESS_BARS=1

env: HF_HUB_DISABLE_PROGRESS_BARS=1


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: winglian/evals
    data_files:
      - hf/ARC-Challenge.jsonl
      - hf/ARC-Easy.jsonl
      - hf/riddle_sense.jsonl
    type: explainchoice:chat
  - path: winglian/evals
    data_files:
      - hf/gsm8k.jsonl
      - hf/winogrande.jsonl
    type: alpaca_chat.load_qa
  - path: winglian/evals
    data_files:
      - custom/n_task.jsonl
      - custom/misconceptions.jsonl
      - custom/context_insensitivity.jsonl
    type: alpaca_chat
  - path: camel-a

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/iss

## #5 mbs 12

In [ ]:
%env HF_HUB_DISABLE_PROGRESS_BARS=1

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: winglian/evals
    data_files:
      - hf/ARC-Challenge.jsonl
      - hf/ARC-Easy.jsonl
      - hf/riddle_sense.jsonl
    type: explainchoice:chat
  - path: winglian/evals
    data_files:
      - hf/gsm8k.jsonl
      - hf/winogrande.jsonl
    type: alpaca_chat.load_qa
  - path: winglian/evals
    data_files:
      - custom/n_task.jsonl
      - custom/misconceptions.jsonl
      - custom/context_insensitivity.jsonl
    type: alpaca_chat
  - path: camel-a

In [ ]:
!pip install git+https://github.com/cphyc/jupyter-notify.git

In [ ]:
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [ ]:
%autonotify -a 30 -o

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/iss

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## #6 mbs 10

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: winglian/evals
    data_files:
      - hf/ARC-Challenge.jsonl
      - hf/ARC-Easy.jsonl
      - hf/riddle_sense.jsonl
    type: explainchoice:chat
  - path: winglian/evals
    data_files:
      - hf/gsm8k.jsonl
      - hf/winogrande.jsonl
    type: alpaca_chat.load_qa
  - path: winglian/evals
    data_files:
      - custom/n_task.jsonl
      - custom/misconceptions.jsonl
      - custom/context_insensitivity.jsonl
    type: alpaca_chat
  - path: camel-a

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/iss

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## #7 mbs 8

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: winglian/evals
    data_files:
      - hf/ARC-Challenge.jsonl
      - hf/ARC-Easy.jsonl
      - hf/riddle_sense.jsonl
    type: explainchoice:chat
  - path: winglian/evals
    data_files:
      - hf/gsm8k.jsonl
      - hf/winogrande.jsonl
    type: alpaca_chat.load_qa
  - path: winglian/evals
    data_files:
      - custom/n_task.jsonl
      - custom/misconceptions.jsonl
      - custom/context_insensitivity.jsonl
    type: alpaca_chat
  - path: camel-a

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/iss

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## #8 No early_stopping_patience

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: /content/llm-playground/models/tiiuae_falcon-40b
base_model_config: /content/llm-playground/models/tiiuae_falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: winglian/evals
    data_files:
      - hf/ARC-Challenge.jsonl
      - hf/ARC-Easy.jsonl
      - hf/riddle_sense.jsonl
    type: explainchoice:chat
  - path: winglian/evals
    data_files:
      - hf/gsm8k.jsonl
      - hf/winogrande.jsonl
    type: alpaca_chat.load_qa
  - path: winglian/evals
    data_files:
      - custom/n_task.jsonl
      - custom/misconceptions.jsonl
      - custom/context_insensitivity.jsonl
    type: alpaca_chat
  - path: camel-a

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)
Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/iss

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## #9

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

# Upload

### Upload checkpoints to HF

In [ ]:
%cd /content/axolotl-trained/falcon-qlora-40b-gsm8k/

In [ ]:
!ls -lhta |grep checkpoint-

In [ ]:
!ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}'

In [ ]:
# ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}' | xargs rm -rf

In [ ]:
!python /workspace/llm-playground/helper/storage.py utensil/axolotl-trained /content/ -u

## Below are ad hoc cells handling issues during training

current out dir:

```
/content/axolotl-trained/falcon-qlora-40b-gsm8k/
```

### Force release VRAM

In [ ]:
# First interupt the kernel, wait a few seconds then run this to kill finetune to release VRAM
!ps aux|grep python|grep finetune|awk '{print $2}'|xargs kill

### Clean the finetuned model and all checkpoints

In [ ]:
# Only run this to start over
!rm -rf /content/axolotl-trained/falcon-qlora-40b-gsm8k/

### Zip the prepared dataset

In [ ]:
!apt install zip
!zip -r last_run_prepared.zip -xi last_run_prepared

### Monitoring GPU

In [ ]:
# Run this in a seperate terminal
!nvitop -m full

### Fix DISK FULL

In [ ]:
%cd /

In [ ]:
!du -d 2 -h|grep G

In [ ]:
!du -d 2 -h /root/.local

In [ ]:
!rm -rf /root/.local/share/Trash/

In [ ]:
!rm -rf /root/.local/share/wandb/

In [ ]:
!rm -rf /root/.cache/wandb/

### Check who is using GPU

In [ ]:
!apt install lsof

In [ ]:
!lsof /dev/nvidia*

### A new bash without tmux etc.

In [ ]:
!bash --norc --noprofile

### Clean up all checkpoints but last one

In [ ]:
!cd /content/axolotl-trained/falcon-qlora-40b-gsm8k/ && ls -lhta |grep checkpoint- | awk '{print $9}'

In [ ]:
!cd /content/axolotl-trained/falcon-qlora-40b-gsm8k/ && ls -lhta |grep checkpoint- | awk 'NR > 0 {print $9}' | xargs rm -rf